In [1]:
import pandas as pd
import datetime
import os
import numpy as np
import hashlib
import gc
os.getcwd()

'/home/jian/celery/Bing_LiveRamp'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            


thismonday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday())
thismonday
last_week_end_saturday=thismonday-datetime.timedelta(days=2)

last_week_end_saturday

datetime.date(2019, 2, 9)

In [3]:
writer_pather="/home/jian/celery/Bing_LiveRamp/output/"

In [4]:
posibble_recent_folder="/home/jian/BigLots/MediaStorm_"+str(last_week_end_saturday)+"/"
daily_files_recent=[x for x in list(recursive_file_gen(posibble_recent_folder)) if "Daily" in x]

list_1_after_201806_2019=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("aily" in x) & (".txt" in x) ]
list_1_after_201806_2019=[x for x in list_1_after_201806_2019 if str(last_week_end_saturday) in x]


daily_files_last_week=daily_files_recent+list_1_after_201806_2019
if len(daily_files_last_week)==1:
    daily_file_last_week=daily_files_last_week[0]
    print("Good to load")
else:
    daily_file_last_week=np.nan
    print("Last week daily data not avaiable")

Good to load


In [5]:
sales_daily_lastweek=pd.read_table(daily_file_last_week,sep="|",dtype=str,usecols=['location_id','customer_id_hashed','transaction_dt','subclass_transaction_amt'])
sales_daily_lastweek=sales_daily_lastweek[~pd.isnull(sales_daily_lastweek['customer_id_hashed'])]
sales_daily_lastweek=sales_daily_lastweek[sales_daily_lastweek['location_id']!="6990"]
sales_daily_lastweek['subclass_transaction_amt']=sales_daily_lastweek['subclass_transaction_amt'].astype(float)
sales_daily_lastweek_agg=sales_daily_lastweek.groupby(['customer_id_hashed','transaction_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index()
sales_daily_lastweek_agg=sales_daily_lastweek_agg.rename(columns={"transaction_dt":"Timestamp","subclass_transaction_amt":"Conversion_Amount"})
sales_daily_lastweek_agg['Timestamp']=sales_daily_lastweek_agg['Timestamp'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())


In [6]:
posibble_recent_Master="/home/jian/BigLots/MediaStorm_"+str(last_week_end_saturday)+"/"
master_files_recent=[x for x in list(recursive_file_gen(posibble_recent_folder)) if "ster" in x]
master_files_recent

['/home/jian/BigLots/MediaStorm_2019-02-09/MediaStormMasterWeekly20190212-122428-267.txt']

In [7]:
posibble_recent_Master="/home/jian/BigLots/MediaStorm_"+str(last_week_end_saturday)+"/"
master_files_recent=[x for x in list(recursive_file_gen(posibble_recent_folder)) if "ster" in x]
if len(master_files_recent)==1:
    master_files_recent=master_files_recent[0]
    print("Good to load")
else:
    master_files_recent=np.nan
    print("Last week Master file not avaiable")
    
recent_date=last_week_end_saturday

Good to load


In [8]:
Master_2019_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("aster" in x) & (".txt" in x) ]
Master_2018_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("aster" in x) & (".txt" in x) ]


weekly_df=pd.DataFrame({"file_path":Master_2019_weekly+Master_2018_weekly})
weekly_df['date']=weekly_df['file_path'].apply(lambda x: x.split("_by_weeks/MediaStorm_")[1][:10])
weekly_df['date']=weekly_df['date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
weekly_df=weekly_df.sort_values("date",ascending=False).reset_index()
del weekly_df['index']

weekly_df=pd.DataFrame({"file_path":master_files_recent,"date":recent_date},index=[0]).append(weekly_df)
weekly_df=weekly_df.drop_duplicates().reset_index()
del weekly_df['index']

In [9]:
data_0=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip",
                     dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code']).drop_duplicates()
print(data_0.shape)


(1873409, 3)


In [10]:
data_1=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStormCustTot-hashed-email.txt",
                     dtype=str,header=None,usecols=[0,1,5])
data_1.columns=['customer_id_hashed','email_address_hash','customer_zip_code']
data_1['customer_id_hashed']=data_1['customer_id_hashed'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())
print(data_1.shape)

(19337597, 3)


In [11]:
data_2 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'MediaStormCustomerTransactionTotals_2018-01-09_2018-03-31.txt',
                     sep = ',',dtype = str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_2.shape)
data_3 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'Existing Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_3.shape)
data_4 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'New Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_4.shape)

(7255226, 3)
(5759051, 3)
(659127, 3)


In [12]:
master_old=data_4.append(data_3).append(data_2).append(data_1).append(data_0).drop_duplicates()
del data_4
del data_3
del data_2
del data_1
del data_0
gc.collect()

42

In [13]:
all_weekly_biweekly_master_file=pd.DataFrame()
for file_path in weekly_df['file_path'].tolist():
    df=pd.read_table(file_path,dtype=str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'],sep="|")
    all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(df)

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(master_old)
print(all_weekly_biweekly_master_file.shape)
print(len(all_weekly_biweekly_master_file['customer_id_hashed'].unique()))

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.drop_duplicates('customer_id_hashed')
print(all_weekly_biweekly_master_file.shape)

(24419168, 3)
24384411
(24384411, 3)


In [14]:
print(sales_daily_lastweek_agg.shape)

sales_daily_lastweek_agg=pd.merge(sales_daily_lastweek_agg,all_weekly_biweekly_master_file,on="customer_id_hashed",how="left").rename(columns={"email_address_hash":"Email_1","customer_zip_code":"Zip"})
sales_daily_lastweek_agg.head(2)


print("Null Email rows excluded: "+str(sales_daily_lastweek_agg[pd.isnull(sales_daily_lastweek_agg['Email_1'])].shape[0]))
sales_daily_lastweek_agg=sales_daily_lastweek_agg[~pd.isnull(sales_daily_lastweek_agg['Email_1'])]
del sales_daily_lastweek_agg['customer_id_hashed']

(1163551, 3)
Null Email rows excluded: 68846


In [15]:
sales_daily_lastweek_agg=sales_daily_lastweek_agg[["Email_1","Zip","Timestamp", "Conversion_Amount"]]
sales_daily_lastweek_agg['Conversion_Amount']=sales_daily_lastweek_agg['Conversion_Amount'].apply(lambda x: np.round(x,2)).astype(str)
sales_daily_lastweek_agg['Conversion_Amount']=sales_daily_lastweek_agg['Conversion_Amount'].apply(lambda x: x.split(".")[0]+"."+x.split(".")[1].ljust(2,"0"))
sales_daily_lastweek_agg['Product_Group']="In_Store"



In [16]:
data_max_date=sales_daily_lastweek_agg['Timestamp'].max()

In [17]:
data_min_date=sales_daily_lastweek_agg['Timestamp'].min()

In [18]:
68846/1163551

0.05916887184145774

In [36]:
local_path=writer_pather+"/BL_LR_BingStoreSales_"+str(data_min_date)+"_"+str(data_max_date)+"_JL_"+str(datetime.datetime.now().date())+".txt"

sales_daily_lastweek_agg.to_csv(local_path,index=False,sep="|")


In [37]:
import paramiko

host = "sftp.liveramp.com" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "Jubaplus2019!" #hard-coded
username = "bing-big-lots" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [38]:
# local_path defined above before saving the local txt
remote_path="/uploads/"+os.path.basename(local_path)
sftp.put(local_path,remote_path)
sftp.close()

Socket exception: Connection timed out (110)
Socket exception: Connection reset by peer (104)
Socket exception: Connection reset by peer (104)


In [5]:
import pandas as pd

test=pd.read_csv("/home/jian/celery/Bing_LiveRamp/output/BL_LR_BingStoreSales_2019-02-03_2019-02-09_JL_2019-02-15.txt",dtype=str,nrows=10,sep="|")

In [6]:
test

,Email_1,Zip,Timestamp,Conversion_Amount,Product_Group
0,8dd56b47142ccb0e94d5ab40548e14db127e368fa64754...,NaN,2019-02-08,8.45,In_Store
1,216b81a8a9eee53938c376a96eaace39016f7787626786...,77840,2019-02-07,4.05,In_Store
2,216b81a8a9eee53938c376a96eaace39016f7787626786...,77840,2019-02-08,16.20,In_Store
3,884c7d27f0df2fdd99383fbe9ca4515281160dd08fb351...,33470,2019-02-09,24.50,In_Store
4,ad222669e6495a48d4391cdce32d76b33cacd7f0796aba...,91710,2019-02-07,6.00,In_Store
5,3030b8eaf1c6c486ccf6752ebab1b9d466e1f0f3c3e3c9...,91780,2019-02-05,16.20,In_Store
6,6b6e75a4d40cbaf02ecfa1c7f83742699189b02bacf275...,22980,2019-02-05,38.20,In_Store
7,ffb897b3251ee884aaffff34293827efe15f3abc679757...,11717,2019-02-09,19.80,In_Store
8,4ea09069a44691691ff86313e06b4163f430dcf1cf0fa3...,29118,2019-02-04,899.98,In_Store
9,10b2ed064ba79950c552bcc5b0b1909d25cd87076cb78a...,45036,2019-02-09,6.50,In_Store
